In [1]:
import torch
import tiktoken

In [16]:
with open('text.txt','r') as f:
    raw_text = f.read()
len(raw_text)    

24118

In [18]:
raw_text[:99]

'As she lay in her berth, staring at the shadows overhead, the rush of the wheels was in her brain, '

In [19]:
tokenizer = tiktoken.get_encoding('gpt2')
vocab = tokenizer.encode(' '.join(raw_text))

In [20]:
len(vocab)

28636